In [1]:
### Guarantor file creation
""" 
Alerts: Main entry point
Developed by CD


Usage:
    python -m src.main
"""

from pathlib import Path
import os

import pandas as pd # type: ignore
import numpy as np # type: ignore

import cdutils.input_cleansing # type: ignore
import cdutils.deduplication # type: ignore
import src.fetch_data
import src.control_panel
import src.flags
import src.flags.credit_score
import src.flags.deposits
import cdutils.joining # type: ignore
import cdutils.loans.calculations # type: ignore
import src.transformation.cleaning
import src.transformation.filtering
import src.transformation.total_exposure
import src.transformation.joining
import src.household
import src.personal_guarantor
import src.flags.past_due
import src.flags.line_utilization
from src._version import __version__


# def main(production_flag: bool=False):
#     if production_flag:
#         BASE_PATH = Path(r'\\00-DA1\Home\Share\Line of Business_Shared Services\Credit_Loan_Review\Alerts\Production')
#         assert "prod" in __version__, (f"Cannot run in production mode without 'prod' in the __version__")
#     else:
#         BASE_PATH = Path('.')

# Database Connection Configuration
data = src.fetch_data.fetch_data()

# Unpack
acctcommon = data['acctcommon'].copy()
acctloan = data['acctloan'].copy()
loans = data['loans'].copy()
househldacct = data['househldacct'].copy()
acctstatistichist = data['acctstatistichist'].copy()
acctloanlimithist = data['acctloanlimithist'].copy()
allroles = data['allroles'].copy()
pers = data['pers'].copy()
viewperstaxid = data['viewperstaxid'].copy()

# Input cleaning
viewperstaxid_schema = {
'persnbr': int,
'taxid': float
}

viewperstaxid = cdutils.input_cleansing.enforce_schema(viewperstaxid, viewperstaxid_schema)


In [2]:
# # Core ETL

loan_data = src.transformation.joining.filter_and_merge_loan_tables(acctcommon, acctloan, loans)
loan_data = src.transformation.total_exposure.append_total_exposure_field(loan_data)
househldacct = src.transformation.cleaning.drop_hh_duplicates(househldacct)
loan_data = src.transformation.joining.append_household_number(loan_data, househldacct)
household_grouping_df = src.household.household_total_exposure(loan_data)
loan_data = src.household.append_household_exposure(loan_data, household_grouping_df)

# Main parameters for loans
loan_data = src.transformation.filtering.filter_to_target_products(
    loan_data,
    creditlimitamt=500000,
    total_exposure=1000000
) # Note that minor list is hard-coded to line of credit minors in the filtering module

# Additional ETL
acctstatistic_output = src.transformation.cleaning.acctstatistichist_cleaning(acctstatistichist, acctcommon)
pd_df = src.flags.past_due.count_pd(acctstatistic_output)
pd30_df = src.flags.past_due.count_pd30(acctstatistic_output)
loan_data = src.flags.past_due.append_pd_info(loan_data, pd_df, pd30_df)
deposit_data = src.flags.deposits.deposit_criteria_testing()
loan_data = src.flags.deposits.append_deposit_data(loan_data, deposit_data)
utilization_data, cleanup_data = src.flags.line_utilization.line_utilization_fetch(loan_data)
loan_data = src.flags.line_utilization.append_line_utilization_data(loan_data, utilization_data, cleanup_data)
inactive_date_df = src.transformation.joining.get_inactive_date(acctloanlimithist)
loan_data = src.transformation.joining.append_inactive_date(loan_data, inactive_date_df)


In [3]:
# # Part of real process
# pers_data = src.personal_guarantor.append_tax_id_to_pers(pers, viewperstaxid)
# pers_data = src.personal_guarantor.append_credit_score(pers_data)
# credit_score_data = src.personal_guarantor.allroles_with_credit_score(allroles, pers_data)



In [4]:
eligible_guar = pd.merge(loan_data, allroles, on='acctnbr',how='left')

In [5]:
eligible_guar

,acctnbr,effdate,mjaccttypcd,product,currmiaccttypcd,bookbalance,loanofficer,ownername,curracctstatcd,contractdate,...,ttm line utilization,cleanup_provision,inactivedate,acctrolecd,acctroledesc,emplroleyn,persnbr,orgnbr,rundate,datelastmaint
0,102199,2025-06-16,CML,FNB - CML Line of Credit,CM62,0,SBLC LOAN OFFICER,NORTHERN RHODE ISLAND CHAMBER OF COMMERCE,ACT,2012-02-09,...,0.000000,0,2026-02-09,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,100957,2025-06-16,CML,FNB - CML Line of Credit,CM62,24442.32,SBLC LOAN OFFICER,GRAPHIC INNOVATIONS INC,ACT,2005-08-15,...,0.165812,1,2025-08-31,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,101199,2025-06-16,CML,FNB - CML Line of Credit,CM62,34169.09,SBLC LOAN OFFICER,ANDERSON-WINFIELD FUNERAL HOME INC,ACT,2003-11-14,...,0.453049,1,2025-07-31,NaN,NaN,NaN,NaN,NaN,NaT,NaT
3,101895,2025-06-16,CML,FNB - CML Line of Credit,CM62,0,SBLC LOAN OFFICER,FORE COURT RACQUET & FITNESS CLUB INC,ACT,2008-03-12,...,0.000000,0,2026-03-12,NaN,NaN,NaN,NaN,NaN,NaT,NaT
4,100135,2025-06-16,CML,FNB - CML Line of Credit,CM62,0,SBLC LOAN OFFICER,"H.B. PRECISION PRODUCTS, INC.",ACT,2002-09-20,...,0.000000,0,2075-01-01,GUAR,Guarantor,N,1120917.0,NaN,2025-06-16,2025-06-16 23:02:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,151150449,2025-06-16,CML,Express Business LOC,CM57,24200,EBL PROGRAM ADMIN,"AMERICA'S BEST DEFENSE, INC.",ACT,2025-02-12,...,0.155358,1,2026-02-12,GUAR,Guarantor,N,1068207.0,NaN,2025-06-16,2025-06-16 23:02:28
395,151125301,2025-06-16,CML,Line of Credit,CM30,0,WILLITTS S. MENDONCA,BEAUPRE ELECTRIC INC,ACT,2024-12-09,...,0.000000,0,2025-12-09,GUAR,Guarantor,N,1063515.0,NaN,2025-06-16,2025-06-16 23:02:28
396,151175546,2025-06-16,CML,Borrowing Base Line of Credit,CM06,200193.5,THOMAS D. KELLY,"COASTLINE ELECTRIC, INC",ACT,2025-04-30,...,0.327660,1,2026-04-30,GUAR,Guarantor,N,1110908.0,NaN,2025-06-16,2025-06-16 23:02:28
397,151175546,2025-06-16,CML,Borrowing Base Line of Credit,CM06,200193.5,THOMAS D. KELLY,"COASTLINE ELECTRIC, INC",ACT,2025-04-30,...,0.327660,1,2026-04-30,GUAR,Guarantor,N,1126887.0,NaN,2025-06-16,2025-06-16 23:02:28


In [6]:


# Remove duplicates for eligible guarantors and create single column dataframe
dedupe_list = [
    {'df':eligible_guar, 'field':'persnbr'}
]

eligible_guar = cdutils.deduplication.dedupe(dedupe_list)
eligible_guar_clean = eligible_guar[['persnbr']].copy()


In [7]:
import cdutils.database.connect # type: ignore
from sqlalchemy import text # type: ignore

def fetch_pers_data():
     # acctcommon
    # engine 1
    wh_pers = text("""
    SELECT 
        * 
    FROM
        OSIBANK.WH_PERS a
    """)

    queries = [
        {'key':'wh_pers', 'sql':wh_pers, 'engine':1},
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data

In [8]:
data = fetch_pers_data()

In [9]:
wh_pers = data['wh_pers'].copy()

In [10]:
def exclude_employees(df: pd.DataFrame, pers: pd.DataFrame) -> pd.DataFrame:
    """
    Exclude employees from the eligible guarantors
    """
    schema_df = {
        'persnbr': str,
    }

    df = cdutils.input_cleansing.enforce_schema(df, schema_df)

    schema_pers = {
        'persnbr': str,
    }
    pers = cdutils.input_cleansing.enforce_schema(pers, schema_pers)

    pers = pers[pers['employeeyn'] != 'Y'].copy()

    merged_df = pd.merge(df, pers, on='persnbr', how='inner')
    merged_df = merged_df[['persnbr']].copy()
    
    return merged_df

In [11]:
eligible_guar_clean = exclude_employees(eligible_guar_clean, wh_pers)

In [12]:
eligible_guar_clean

,persnbr
0,1120917
1,1123487
2,1124298
3,1124299
4,1124300
...,...
311,1171951
312,1068207
313,1063515
314,1110908


In [13]:
PERMISSIONS_INPUT = Path(r"\\00-da1\Home\Share\Data & Analytics Initiatives\Project Management\Credit_Loan_Review\Alerts\Production\assets\xactus\pfs_permission.csv")


import os
def permission_layer(df, permissions_csv_path):
    """
    Filter dataframe based on permissions and return filtered data plus exceptions.
    
    Args:
        df (pd.DataFrame): Input dataframe containing persnbr column
        permissions_csv_path (str): Absolute path to permissions CSV file
    
    Returns:
        tuple: (filtered_df, exceptions_df)
            - filtered_df: Records with valid permissions (inner join result)
            - exceptions_df: Records from input df without permissions
    
    Raises:
        FileNotFoundError: If permissions CSV doesn't exist
        KeyError: If persnbr column missing from either dataframe
        AssertionError: If duplicate persnbr found in either dataframe
        ValueError: If either dataframe is empty after preprocessing
    """
    
    # Validate permissions file exists
    if not os.path.exists(permissions_csv_path):
        raise FileNotFoundError(f"Permissions file not found: {permissions_csv_path}")
    
    # Read permissions CSV
    permissions_df = pd.read_csv(permissions_csv_path)

   
    # validate required columns exist
    if 'persnbr' not in df.columns:
        raise KeyError("column 'persnbr' not found in input dataframe")
    
    if 'persnbr' not in permissions_df.columns:
        raise KeyError("Column 'persnbr' not found in permissions CSV")
    
    # Drop rows where persnbr is null from input df
    df_clean = df.dropna(subset=['persnbr']).copy()
    
    if df_clean.empty:
        raise ValueError("Input dataframe is empty after removing null persnbr values")
    
    if permissions_df.empty:
        raise ValueError("Permissions dataframe is empty")
    
    # Check for duplicate persnbr in both dataframes
    if df_clean['persnbr'].duplicated().any():
        duplicates = df_clean[df_clean['persnbr'].duplicated(keep=False)]['persnbr'].unique()
        raise AssertionError(f"Duplicate persnbr found in input dataframe: {duplicates}")
    
    if permissions_df['persnbr'].duplicated().any():
        duplicates = permissions_df[permissions_df['persnbr'].duplicated(keep=False)]['persnbr'].unique()
        raise AssertionError(f"Duplicate persnbr found in permissions dataframe: {duplicates}")


    schema_df_clean = {
        'persnbr': str,
    }

    df_clean = cdutils.input_cleansing.enforce_schema(df_clean, schema_df_clean)

    schema_permissions_df = {
        'persnbr': str,
    }

    permissions_df = cdutils.input_cleansing.enforce_schema(permissions_df, schema_permissions_df)
    
     
    # Perform outer merge to identify all cases
    merged_df = df_clean.merge(
        permissions_df, 
        on='persnbr', 
        how='outer', 
        indicator=True,
        suffixes=('', '_perm')
    )
    
    # Filter for records that exist in both (inner join result)
    filtered_df = merged_df[merged_df['_merge'] == 'both'].drop('_merge', axis=1)
    
    # Identify exceptions: records in input df but not in permissions
    exceptions_df = merged_df[merged_df['_merge'] == 'left_only'].drop('_merge', axis=1)
    
    # Clean up exceptions_df by removing permission columns (they'll be NaN anyway)
    perm_cols = [col for col in exceptions_df.columns if col.endswith('_perm')]
    exceptions_df = exceptions_df.drop(perm_cols, axis=1)
    
    return filtered_df, exceptions_df


In [14]:
final_eligible, exceptions_df = permission_layer(eligible_guar_clean, PERMISSIONS_INPUT)
## Cleaning stages to get it to ready to put on the template

In [15]:
final_eligible

,persnbr
0,1000637
1,1000763
2,1012786
3,1021325
4,1025471
...,...
326,1172244
327,1172253
329,1173261
330,1173569


In [16]:
exceptions_df

,persnbr
103,1047381
113,1053217
121,1059899
208,1076576
244,1120883
248,1122570
249,1122571
250,1123487
261,1124485
262,1124509


In [17]:
def append_pers_name(df, pers):
    """
    Append person name
    """
    schema_df = {
        'persnbr': str,
    }

    df = cdutils.input_cleansing.enforce_schema(df, schema_df)

    schema_pers = {
        'persnbr': str,
    }
    pers = cdutils.input_cleansing.enforce_schema(pers, schema_pers) 

    dedupe_list = [
        {'df':pers, 'field':'persnbr'}
    ]

    pers = cdutils.deduplication.dedupe(dedupe_list)

    merged_df = pd.merge(df, pers, on='persnbr')

    return merged_df

In [18]:
final_eligible_names = append_pers_name(final_eligible, pers)
exceptions_names = append_pers_name(exceptions_df, pers)


In [19]:
final_eligible_names

,persnbr,firstname,lastname,datebirth,deathnotificationdate
0,1000637,ERIC,ANDERSON,1978-05-13,NaT
1,1000763,LUKE,ANDERSON,1982-06-10,NaT
2,1012786,MICHAEL,BRIGGS,1967-09-08,NaT
3,1021325,PAMELA,DUMAS,1965-12-27,NaT
4,1025471,TIMOTHY,DUBUC,1971-01-24,NaT
...,...,...,...,...,...
300,1172244,CAROLINA,MASSOTE,1989-06-25,NaT
301,1172253,FABIANA,MASSOTE,1977-11-13,NaT
302,1173261,LEO,DUBE,1966-07-09,NaT
303,1173569,LISA,PEARSON,1986-07-17,NaT


In [20]:
exceptions_names

,persnbr,firstname,lastname,datebirth,deathnotificationdate
0,1047381,NANCY,DAWSON,1939-10-31,NaT
1,1053217,GLORIA,JORGE,1955-04-17,NaT
2,1059899,CLIFFORD,BODGE,1935-01-16,NaT
3,1076576,BRIAN,THOMASSET,1956-01-09,NaT
4,1120883,ALFRED,COSTANTINO,1948-09-03,NaT
5,1122570,OZCAN,ETEMAN,1964-03-20,NaT
6,1122571,RUHSAR,ETEMAN,1963-07-16,NaT
7,1123487,LAWRENCE,PARENTEAU,1969-01-16,NaT
8,1124485,KATHLEEN,GIORGI,1951-05-10,NaT
9,1124509,DANIEL,PIERCE,1966-06-19,NaT


In [21]:
schema_exceptions = {
    'persnbr': str,
}

exceptions_names = cdutils.input_cleansing.enforce_schema(exceptions_names, schema_exceptions)

schema_eligible_guar = {
    'persnbr': str,
}
eligible_guar = cdutils.input_cleansing.enforce_schema(eligible_guar, schema_eligible_guar) 


In [22]:
exceptions_loan_data = pd.merge(exceptions_names, eligible_guar, on='persnbr', how='left')

In [23]:
exceptions_loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 46 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   persnbr                 11 non-null     object        
 1   firstname               11 non-null     object        
 2   lastname                11 non-null     object        
 3   datebirth               11 non-null     datetime64[ns]
 4   deathnotificationdate   0 non-null      datetime64[ns]
 5   acctnbr                 11 non-null     int64         
 6   effdate                 11 non-null     datetime64[ns]
 7   mjaccttypcd             11 non-null     object        
 8   product                 11 non-null     object        
 9   currmiaccttypcd         11 non-null     object        
 10  bookbalance             11 non-null     object        
 11  loanofficer             11 non-null     object        
 12  ownername               11 non-null     object      

In [24]:
exceptions_loan_data = exceptions_loan_data[['persnbr','firstname','lastname','datebirth','product','loanofficer','ownername']].copy()
exceptions_loan_data = exceptions_loan_data.rename(columns={'ownername':'Customer Name'}).copy()

In [25]:
exceptions_loan_data

,persnbr,firstname,lastname,datebirth,product,loanofficer,Customer Name
0,1047381,NANCY,DAWSON,1939-10-31,Borrowing Base Line of Credit,ANDREW J. OMER,CENTRAL TOOLS INC
1,1053217,GLORIA,JORGE,1955-04-17,Line of Credit,SBLC LOAN OFFICER,JORGE DRYWALL CO INC
2,1059899,CLIFFORD,BODGE,1935-01-16,Line of Credit,KEVIN M. MCCARTHY,TAUNTON STOVE COMPANY INC
3,1076576,BRIAN,THOMASSET,1956-01-09,Line of Credit,SBLC LOAN OFFICER,"B.R. THOMASSET, INC."
4,1120883,ALFRED,COSTANTINO,1948-09-03,Line of Credit,SBLC LOAN OFFICER,"ALCO PROPERTIES, LLC"
5,1122570,OZCAN,ETEMAN,1964-03-20,FNB - SBA Line of Credit,SBLC LOAN OFFICER,ETEMAN LLC
6,1122571,RUHSAR,ETEMAN,1963-07-16,FNB - SBA Line of Credit,SBLC LOAN OFFICER,ETEMAN LLC
7,1123487,LAWRENCE,PARENTEAU,1969-01-16,FNB - SBA Line of Credit,SBLC LOAN OFFICER,"PARENTEAU TRUCK & EQUIPMENT, LLC"
8,1124485,KATHLEEN,GIORGI,1951-05-10,FNB - CML Line of Credit,SBLC LOAN OFFICER,FLOORS BEAUTIFUL BY DEE
9,1124509,DANIEL,PIERCE,1966-06-19,FNB - SBA Line of Credit,SBLC LOAN OFFICER,PIERCE PACKAGING PRODUCTS INC


In [26]:
# Creating the final output for the Xactus Extract


In [27]:
import cdutils.append_taxid # type: ignore

xactus_outbound = cdutils.append_taxid.append_tax_id_to_pers(final_eligible_names)

In [28]:
xactus_outbound

,persnbr,firstname,lastname,datebirth,deathnotificationdate,taxid
0,1000637,ERIC,ANDERSON,1978-05-13,NaT,035524004
1,1000763,LUKE,ANDERSON,1982-06-10,NaT,030625641
2,1012786,MICHAEL,BRIGGS,1967-09-08,NaT,015649402
3,1021325,PAMELA,DUMAS,1965-12-27,NaT,011589245
4,1025471,TIMOTHY,DUBUC,1971-01-24,NaT,029662889
...,...,...,...,...,...,...
300,1172244,CAROLINA,MASSOTE,1989-06-25,NaT,013889008
301,1172253,FABIANA,MASSOTE,1977-11-13,NaT,634775258
302,1173261,LEO,DUBE,1966-07-09,NaT,036422885
303,1173569,LISA,PEARSON,1986-07-17,NaT,022681983


In [29]:
import cdutils.primary_address # type: ignore

xactus_outbound = cdutils.primary_address.append_primary_address(xactus_outbound)

In [30]:
xactus_outbound

,persnbr,firstname,lastname,datebirth,deathnotificationdate,taxid,text1,cityname,statecd,zipcd
0,1000637,ERIC,ANDERSON,1978-05-13,NaT,035524004,66 WAUREGAN RD,BROOKLYN,CT,06234
1,1000763,LUKE,ANDERSON,1982-06-10,NaT,030625641,75 ROUND ST,TAUNTON,MA,02780
2,1012786,MICHAEL,BRIGGS,1967-09-08,NaT,015649402,660 PAINE RD,NORTH ATTLEBORO,MA,02760
3,1021325,PAMELA,DUMAS,1965-12-27,NaT,011589245,26 MILL ST,BERKLEY,MA,02779
4,1025471,TIMOTHY,DUBUC,1971-01-24,NaT,029662889,102 MORGAN DR,TAUNTON,MA,02780
...,...,...,...,...,...,...,...,...,...,...
300,1172244,CAROLINA,MASSOTE,1989-06-25,NaT,013889008,27332 HOLLYBROOK TRAIL,WESLEY CHAPEL,FL,33544
301,1172253,FABIANA,MASSOTE,1977-11-13,NaT,634775258,15 WINIFRED ROAD,FRAMINGHAM,MA,01701
302,1173261,LEO,DUBE,1966-07-09,NaT,036422885,10 WHITE OAK LN,NORTH DARTMOUTH,MA,02747
303,1173569,LISA,PEARSON,1986-07-17,NaT,022681983,13 11TH AVE,WAREHAM,MA,02571


In [31]:
xactus_outbound['addrnbr'] = ""
xactus_outbound['Pre-Directional'] = ""
xactus_outbound['Post-Directional'] = ""
xactus_outbound['Street Type'] = ""
xactus_outbound['Maternal Name'] = ""

In [32]:
xactus_outbound = xactus_outbound[['firstname','lastname','persnbr','taxid','addrnbr','Pre-Directional','text1','Post-Directional','Street Type','cityname','statecd','zipcd','Maternal Name']].copy()

In [33]:
XACTUS_OUTBOUND_RAW = Path(r"\\00-da1\Home\Share\Data & Analytics Initiatives\Project Management\Credit_Loan_Review\Alerts\Production\Xactus\Staging\xactus_outbound_raw_data.csv")
xactus_outbound.to_csv(XACTUS_OUTBOUND_RAW)

In [34]:
xactus_outbound.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   firstname         305 non-null    object
 1   lastname          305 non-null    object
 2   persnbr           305 non-null    object
 3   taxid             304 non-null    object
 4   addrnbr           305 non-null    object
 5   Pre-Directional   305 non-null    object
 6   text1             304 non-null    object
 7   Post-Directional  305 non-null    object
 8   Street Type       305 non-null    object
 9   cityname          304 non-null    object
 10  statecd           304 non-null    object
 11  zipcd             304 non-null    object
 12  Maternal Name     305 non-null    object
dtypes: object(13)
memory usage: 31.1+ KB


In [ ]:
### REST OF SCRIPT

In [ ]:
# Control panel for additional parameters
loan_data = src.control_panel.criteria_flags(
    loan_data,
    credit_score_data,
    score_floor = 680, # Minimum Credit Score for any guarantor related to the loan
    score_decrease = -0.1, # Decrease of credit score since prior period
    ttm_pd_amt = 3, # Times in trailing 12 Months Past Due (15-29)
    ttm_pd30_amt = 1, # Times in trailing 12 Months Past Due (30+)
    ttm_overdrafts = 5, # Number of days with an overdrawn balance in trailing 12 Months
    deposit_change_pct = -.3, # Aggregate deposits in relationship decrease (-30% loss)
    min_deposits = 50000, # Minimum deposits to be eligible to flag deposit decrease
    utilization_limit = .7 # Line Utilization flag for trailing 12 months
)

# Consolidation of the columns necessary
final_df = loan_data[['acctnbr','effdate','ownername','product','loanofficer','inactivedate','Net Balance','Net Available','Net Collateral Reserve','cobal','creditlimitamt','Total Exposure_hhgroup','ttm_pd','ttm_pd30','TTM Days Overdrawn','3Mo_AvgBal','TTM_AvgBal','Deposit Change Pct','ttm line utilization','cleanup_provision','riskratingcd','past_due_flag','ttm_overdrafts_flag','deposit_change_flag','ttm_utilization_flag','score_flag','passed_all_flag']]

# Writing output
ALERTS_PATH = BASE_PATH / Path('./Output/alerts.xlsx')
final_df.to_excel(ALERTS_PATH, index=False, engine='openpyxl', sheet_name='Sheet1')

CREDSCORE_PATH = BASE_PATH / Path('./Output/credit_score_detail.xlsx')
credit_score_data.to_excel(CREDSCORE_PATH, index=False, engine='openpyxl', sheet_name='Sheet1')
    
print('Execution Complete!')

if __name__ == "__main__":
print(f"Starting alerts [{__version__}]")
# main(production_flag=True)
main()
print("Complete!")

    




